In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout



In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 32
path = "/kaggle/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"

In [ ]:
# cat tap train : Validation + scale anh
datagen = ImageDataGenerator(
    rescale=1./255, # chuyen doi gia tri [0,255] ve [0,1]
    validation_split=0.2) # chia du lieu thanh tap train va validation

In [ ]:
train_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE ),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset=('training') # chi lay tap train 
)

val_data = datagen.flow_from_directory(
    path,
    target_size=(IMG_SIZE, IMG_SIZE ),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset=('validation') # chi lay tap validation
)

# KHAI BAO MO HINH - FIT

In [ ]:
model = Sequential(
    [
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2, 2)),

        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),

        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        
        Flatten(), 
        Dense(512, activation='relu'),# fully connected layer
        Dropout(0.5), #dropout de tranh overfitting
        Dense(29, activation='softmax') # 29 lop dau ra

    ]
)

model.summary()

In [ ]:
# huan luyen
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

# Save model

In [ ]:
model.save('asl_alphabet_model.h5') # .h5 dung de load lai va du doan sau nay
print("Model saved to asl_alphabet_model.h5")

# danh gia mo hinh

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f'Accuracy: { accuracy:.4f}')
print(f'Loss: {loss:.4f}')


# du doan

In [ ]:
# load model
model = keras.models.load_model("/kaggle/working/asl_alphabet_model.h5")
print("Model loaded from asl_alphabet_model.h5")


In [ ]:
#chuan bi label (chu cai)
class_names = list(train_data.class_indices.keys())
print(class_names)

In [ ]:
# load 1 hinh de xu li
img_path = path + '/test.jpg' # duong dan den anh test
# img_path = input("Enter the path of the image to predict: ")
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img)
img_array= img_array / 255.0  # chuan hoa anh
img_array = np.expand_dims(img_array, 0)  # them kich thuoc batch



In [ ]:
# du doan
predictions = model.predict(img_array)
predicted_index = np.argmax(predictions)
predicted_class = class_names[predicted_index]

# print % cac lop du doan
print(predictions)
print(f'predicted label: {predicted_label}')